In [ ]:
import os
import shutil
import re

In [ ]:
# Source and destination directories
source_dir = "C://Users//sg16g//Music//suno"  
destination_dir = "C://Users//sg16g//Music//dataset//suno" 
duplicates_dir = "C://Users//sg16g//Music//dataset//suno//duplicates" 

# Ensure destination directory exists
os.makedirs(destination_dir, exist_ok=True)
os.makedirs(duplicates_dir, exist_ok=True)

In [ ]:
# mass migration

# Track files to detect duplicates
existing_files = set(os.listdir(destination_dir))  # Store existing file names

# Walk through all subdirectories
for root, _, files in os.walk(source_dir):

  if root != source_dir:  # If we're inside a subfolder
    print(f"Entering subfolder: {root}")

  for file in files:

    new_name = re.sub(r'^\d+\s*-\s*', '', file)

    # Full file paths
    old_path = os.path.join(root, file)
    new_path = os.path.join(destination_dir, new_name)

    # Check if the file already exists in the destination
    if new_name in existing_files:
      # Check if the file is already in the destination folder
      duplicate_path = os.path.join(duplicates_dir, new_name)

      # If file exists, rename with a number suffix and move it to the duplicates folder
      base, ext = os.path.splitext(new_name)
      count = 1
      while os.path.exists(duplicate_path):  # Check for conflicts in duplicates folder
        duplicate_path = os.path.join(duplicates_dir, f"{base}_{count}{ext}")
        count += 1

      shutil.move(old_path, duplicate_path)
      print(f"Duplicate found: {file} → Moved to duplicates folder as {os.path.basename(duplicate_path)}.")
    else:
      shutil.move(old_path, new_path)
      existing_files.add(new_name)  # Track moved files

print("All files moved successfully!")

In [ ]:
# Get file sizes in destination folder
destination_files = {f: os.path.getsize(os.path.join(destination_dir, f)) for f in os.listdir(destination_dir)}

# Process duplicates
for file in os.listdir(duplicates_dir):
    old_path = os.path.join(duplicates_dir, file)

    # Rename file (remove leading numbers and hyphen)
    new_name = re.sub(r'^\d+\s*-\s*', '', file)
    new_path = os.path.join(destination_dir, new_name)

    if new_name in destination_files:
        dup_size = os.path.getsize(old_path)
        dest_size = destination_files[new_name]

        if dup_size != dest_size:
            # File exists but has a different size → Rename with a number suffix
            base, ext = os.path.splitext(new_name)
            count = 1
            while os.path.exists(new_path):
                new_path = os.path.join(destination_dir, f"{base}_{count}{ext}")
                count += 1

            shutil.move(old_path, new_path)
            print(f"Renamed & moved: {file} → {os.path.basename(new_path)} (Different size)")
        else:
            print(f"File {new_name} is identical in size → Keeping in duplicates folder.")
    else:
        shutil.move(old_path, new_path)
        print(f"Renamed & moved: {file} → {new_name} (New file)")

print("Duplicate processing completed.")

In [ ]:
# File counter for renaming
count = 1

# Walk through all files in source folder (including subfolders)
for root, _, files in os.walk(destination_dir):

    for file in files:
        ext = os.path.splitext(file)[1]  # Get file extension
        new_name = f"S{count:02d}R{ext}"  # Rename to "song count.extension"
        old_path = os.path.join(root, file)
        new_path = os.path.join(root, new_name)

        # Ensure the new name doesn't already exist in the same folder
        if os.path.exists(new_path):
            print(f"Skipping {file}: {new_name} already exists.")
        else:
            os.rename(old_path, new_path)
            count += 1  # Increment the count for the next file
            print(f"Renamed: {file} → {new_name}")

print("Renaming completed.")
